# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Problema 1

### Descrição do Problema

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    1. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot”  $$1..T\,$$ $$(\text{hora},\text{dia})$$.  
    2.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto. 
    
    
    ( falta deixar isto mais clean)
    

### Código Python

In [ ]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

Criação da matriz de alocação:

In [ ]:
X = {}
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                X[p,c,s,t] = horario.BoolVar(f'X[{p}][{c}][{s}][{t}]')

O lider participa em todas as reuniões do projeto:

$$
\forall_{p< P,c< C,s< S,t<T }  \quad x_{p,l_p,s,t} \ge x_{p,c,s,t}
$$

In [ ]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,l[p],s,t] >= X[p,c,s,t])


O projeto tem mínimo (“quorum”) de 50% do total de colaboradores do projeto:
$$
\forall_{p< P,s< S,t< T} \quad \sum_{c< C} x_{p,c,s,t} \ge \frac{\sum_{c< C} col_{p,c}}{2}
$$

In [ ]:
for p in range(1,P+1):
    for s in range(1,S+1):
        for t in range(1,T+1):
            horario.Add(sum([ X[p,c,s,t] for c in range(1,C+1)]) >= sum([ col[p,c] for c in range(1,C+1) ])/2)
                

Verificar se o colaborador tem disponibilidade:
$$
\forall_{c< C,t< T} \quad d_{c,t} = 0 \quad \implies \quad \sum_{p<P,s< S} x_{p,c,s,t} = 0
$$

In [ ]:
for c in range(1,C+1):
    for t in range(1,T+1):
        if d[c,t] == 0:
            horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) == 0)

Número Mínimo de reuniões semanais:
$$
\forall_{p< P} \quad \sum_{s< S,t< T} x_{p,l_p,s,t} \ge m_p
$$

In [ ]:
for p in range(1,P+1):
    horario.Add(sum([ X[p,l[p],s,t] for c in range(1,C+1) for s in range(1,S+1) for t in range(1,T+1)]) >= m[p])

Colaboradores não podem participar em reuniões de projetos que não estão associados:
$$
\forall_{p< P,c< C,s< S,t< T} \quad x_{p,c,s,t} \le col_{p,c}
$$

In [ ]:
for p in range(1,P+1):
    for c in range(1,C+1):
        for s in range(1,S+1):
            for t in range(1,T+1):
                horario.Add(X[p,c,s,t] <= col[p,c])

Um colaborador não pode participar em duas reuniões ao mesmo tempo:
$$
\forall_{c< C, t< T} \quad\sum_{p< P, s< S} x_{p,c,s,t} \le 1
$$

In [ ]:
for c in range(1,C+1):
    for t in range(1,T+1):
        horario.Add(sum([ X[p,c,s,t] for p in range(1,P+1) for s in range(1,S+1)]) <= 1)

In [ ]:
# S = Int
# C = Int
# T = (hora, dia)
# P = ([conjunto de colaboradores], lider, número de reuniões semanais)

# Maximizar o número de reuniões efetivamente realizadas
# Minimizar o número médio de reuniões por participante.